In [29]:
import os 
import json
import pandas as pd
import traceback
#python dotenv helps in creating variable from .env to env variable


In [99]:
from dotenv import load_dotenv
load_dotenv()
key=os.getenv("myKey")


In [52]:
from langchain_groq import ChatGroq
llm=ChatGroq(
    api_key=key,
    model="llama-3.3-70b-versatile",
    temperature=0.5
)

In [32]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
import PyPDF2



In [74]:
response_json = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "correct answer"
  },
  "3": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "correct answer"
  }
}



In [75]:
TEMPLATE = """
Text: {text}

You are an expert MCQ creator. From the above text, create {number} multiple-choice questions for the subject {subject}, written in a {tone} tone.  
- Questions must be unique and aligned with the text.  
- Use the response_json format as a guide.  
- Output exactly {number} questions.  

### response_json
{response_json}
"""


In [76]:
quiz_generation_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [77]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [ ]:
Template2 = """
You are an English grammarian and writer. Given a multiple-choice quiz for {subject},  
- Evaluate the question complexity in exactly 50 words.  
- If questions do not match student abilities, revise them and adjust the tone appropriately.  

Quiz_MCQ:
{quiz}

Provide expert feedback and improvements:
"""


In [78]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=Template2)
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [80]:


generate_evaluation_chain = SequentialChain(
    chains=[quiz_chain, review_chain],  
    input_variables=['text','number','subject','tone','response_json'],
    output_variables=['quiz','review'],
    verbose=True
)


In [64]:
with open('/Users/nameesh/mcq_generator/experiment/data.txt','r')as file:
    Text=file.read()

In [25]:
#serializing the python dictionary into json-formatted string
json.dumps(response_json)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [65]:
print(Text)

Machine Learning (ML) is a branch of artificial intelligence that develops algorithms able to learn from data and generalize to new situations without explicit programming. Grounded in statistics, optimization, and data mining, ML is widely applied in natural language processing, computer vision, speech recognition, medicine, agriculture, and business analytics. Deep learning, a subset of ML using neural networks, has surpassed many traditional approaches in performance and drives much of modern AI progress.

Historically, ML traces back to Donald Hebb’s theories of neural structures in 1949 and Arthur Samuel’s checkers program in the 1950s. Research expanded in the 1960s–80s with reinforcement learning, symbolic AI, and early neural networks, but it was the rediscovery of backpropagation in the 1980s that revived interest. By the 1990s, ML became recognized as its own discipline, shifting from symbolic reasoning to data-driven, statistical methods aimed at solving practical problems.


In [81]:
#if we want to see the tokens usage of our model we can use get_openai_callback
response=generate_evaluation_chain({
    "text":Text,
    "number":5,
    "subject":'data science',
    "tone":"simple",
    "response_json":json.dumps(response_json)
})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Machine Learning (ML) is a branch of artificial intelligence that develops algorithms able to learn from data and generalize to new situations without explicit programming. Grounded in statistics, optimization, and data mining, ML is widely applied in natural language processing, computer vision, speech recognition, medicine, agriculture, and business analytics. Deep learning, a subset of ML using neural networks, has surpassed many traditional approaches in performance and drives much of modern AI progress.

Historically, ML traces back to Donald Hebb’s theories of neural structures in 1949 and Arthur Samuel’s checkers program in the 1950s. Research expanded in the 1960s–80s with reinforcement learning, symbolic AI, and early neural networks, but it was the rediscovery of backpropagation in the 1980s that revived interest. By the 1990s, ML became recognized as its own disciplin

In [82]:
quiz=response.get("quiz")

In [88]:
cleaned = quiz.replace("### response_json", "").strip("` \n")

In [89]:
quiz = json.loads(cleaned)

In [ ]:
quiz


{'1': {'mcq': 'What branch of artificial intelligence develops algorithms to learn from data and generalize to new situations?',
  'options': {'a': 'Deep Learning',
   'b': 'Machine Learning',
   'c': 'Natural Language Processing',
   'd': 'Computer Vision'},
  'correct': 'b'},
 '2': {'mcq': 'In which decade did Machine Learning become recognized as its own discipline?',
  'options': {'a': '1960s', 'b': '1970s', 'c': '1980s', 'd': '1990s'},
  'correct': 'd'},
 '3': {'mcq': 'Who is associated with theories of neural structures in 1949?',
  'options': {'a': 'Arthur Samuel',
   'b': 'Donald Hebb',
   'c': 'Alan Turing',
   'd': 'Marvin Minsky'},
  'correct': 'b'},
 '4': {'mcq': 'What method did the field of Machine Learning shift to by the 1990s?',
  'options': {'a': 'Symbolic reasoning',
   'b': 'Data-driven, statistical methods',
   'c': 'Rule-based systems',
   'd': 'Expert systems'},
  'correct': 'b'},
 '5': {'mcq': 'What is the name of the subset of Machine Learning that uses neural 

In [94]:
#converting data into a list format so that we can convert it into a dataframe
quiz_table_data = []
for key, values in quiz.items():
    mcq = values['mcq']
    options = "|".join(
        [
            f"{option}:{option_value}"
            for option, option_value in values['options'].items()
        ]
    )
    correct = values['correct']
    quiz_table_data.append({"MCQ": mcq, "OPTIONS": options, "CORRECT": correct})


In [97]:
quiz=pd.DataFrame(quiz_table_data)

In [98]:
quiz.to_csv("machine_learning_quiz",index=False)